In [19]:

from langchain.llms import CTransformers
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator


In [46]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer
model_path = "/home/ubuntu/work/dl/models/llama-2-7b-chat.ggmlv3.q8_0.bin"
# llm_model = CTransformers(
#     model = model_path,
#     model_type = 'llama'
#     # config = {
#     #     'max_new_tokens': 256,
#     #     'temperature': 0.1
#     # } 
# )

llm_model = AutoModelForCausalLM.from_pretrained(model_path,model_type='llama')

In [10]:
# llm_model([
#     SystemMessage(content="you are a comedian ai assistant"),
#     HumanMessage(content="Please provide some comedy punchline on AI")
# ])

In [48]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    # @validator("setup")
    # def question_ends_with_question_mark(cls, field):
    #     if field[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm_model | parser

chain.invoke({"query": joke_query})

TypeError: object of type 'StringPromptValue' has no len()

In [49]:
from langchain_core.output_parsers import JsonOutputParser

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm_model | parser

chain.invoke({"query": joke_query})

for s in chain.stream({"query": joke_query}):
    print(s)

TypeError: object of type 'StringPromptValue' has no len()

In [40]:
joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm_model | parser

chain.invoke({"query": joke_query})

{'query': 'What is the meaning of life?',
 'joke': 'Why was the math book sad? Because it had too many problems.'}

In [ ]:
from typing import List
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Give 5 movie names by act by."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm_model | parser

chain.invoke({"query": actor_query})

Actor(name='Leonardo DiCaprio', film_names=['Titanic', 'The Wolf of Wall Street', 'Catch Me If You Can', 'Inception', 'The Revenant'])